# Importing Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
import numpy as np
from tqdm import tqdm
from TextToConcept import TextToConcept
import os
import torchvision.transforms as transforms
import torch.nn as nn

/home/umair.nawaz/.conda/envs/t2c/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# In case if multiple GPUs
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # GPU I

# Check if CUDA is available and list available CUDA devices
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs available: ", torch.cuda.device_count())
    print("GPU Name:", torch.cuda.get_device_name())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

CUDA is available. Number of GPUs available:  1
GPU Name: NVIDIA GeForce RTX 4090
Using device: cuda


In [3]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

In [4]:
std_transform_without_normalization = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor()])


std_transform_with_normalization = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)])

#
In this part, we load the DINO model. We also use ``forward_features(x)`` that takes a tensor as the input and outputs the representation (features) of input $x$ when it is passed through the model and ``get_normalizer``, which is the normalizer that the models uses to preprocess the input.

In [5]:
# Local path to the cloned dino repository
local_repo_path = '../Weights/dino_pretrain.pth'  # Replace with your local path

model = torch.hub.load('facebookresearch/dino:main', 'dino_resnet50', pretrained=True)
model.load_state_dict(torch.load(local_repo_path , weights_only=True))

def forward_features(x):
    # Assuming the model returns the last layer's output which might include a classification token or similar
    features = model(x)  # This call should be adjusted based on the actual output format
    # You might need to process the features here, e.g., selecting the right tensor or applying global pooling
    return features.squeeze()  # Adjust this based on the actual structure of your features


# Attach the custom forward method to the model
model.forward_features = forward_features

model.get_normalizer = torchvision.transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
model.has_normalizer = True

Using cache found in /home/umair.nawaz/.cache/torch/hub/facebookresearch_dino_main
/home/umair.nawaz/.conda/envs/t2c/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/umair.nawaz/.conda/envs/t2c/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<b>Initiating Text-To-Concept Object</b><br>
In this section, we initiate ``TextToConcept`` object which turns the vision encoder into a model capable of integrating language and vision.

In [6]:
text_to_concept = TextToConcept(model, 'Dino')

<b>Loading the Linear Aligner</b><br>

In [7]:
text_to_concept.load_linear_aligner('/home/umair.nawaz/Research_Work/Submission/AgriCLIP/Weights/Aligned_Models/Agri_Dino_aligner_DPT_CPT.pth')

/home/umair.nawaz/Research_Work/Submission/AgriCLIP/AgriCLIP_alignment/LinearAligner.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aligner_dict = torch.load(path_to_l

### Zero-shot Classifier

In [8]:

from torchvision.datasets import ImageFolder

# Specify the directory where images are stored
directory_path = '/home/umair.nawaz/Research_Work/Main-DATA/My_Surgical/downstream-final/Banana Deficiency'

# Create the dataset using the ImageFolder class
dataset = ImageFolder(root=directory_path,
                            transform=std_transform_with_normalization)


####

# 50% of images are fairly enough.
# num_of_samples = int(0.5 * len(cifar_dataset))
# cifar_dset = torch.utils.data.Subset(cifar_dataset, np.random.choice(np.arange(len(cifar_dataset)), num_of_samples, replace=False))

####

loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=8)

# Get class name mappings
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}
# Convert dictionary values to a list, sorted by key
class_names_list = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
print(class_names_list)

['boron', 'calcium', 'healthy', 'iron', 'magnesium', 'manganese', 'potassium', 'sulphur', 'zinc']


In [9]:
cifar_zeroshot_classifier = text_to_concept.get_zero_shot_classifier(class_names_list,
                                                                     prompts=['a photo contain {} deficiency'])

In [11]:

correct, total = 0, 0
with torch.no_grad():
    for data in tqdm(loader):
        x, y = data[:2]
        x = x.to(device)

        try:
            # Attempt to compute the outputs
            outputs = cifar_zeroshot_classifier(x).detach().cpu()
            _, predicted = outputs.max(1)  # Find the index of the max log-probability
            total += y.size(0)
            correct += predicted.eq(y).sum().item()
        except IndexError as e:
            print(f"Error processing batch: {e}")
            print(f"x.shape: {x.shape}, y: {y}")
            continue  # Skip this batch and continue with the next

  0%|          | 0/97 [00:00<?, ?it/s]

100%|██████████| 97/97 [00:03<00:00, 30.33it/s]


In [12]:
f'Zeroshot Accuracy: {100.*correct/total:.2f}'

'Zeroshot Accuracy: 23.55'